In [ ]:
!pip install torch lightning numpy kaggle wandb

In [ ]:
from google.colab import files

# Carica il file kaggle.json
files.upload()


In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d ealaxi/paysim1
!unzip paysim1.zip

In [ ]:
import pandas as pd, sys, plotly.graph_objects as go, plotly.express as px, numpy as np, torch, random as rnd, torch.nn as nn, lightning as l, wandb as wndb
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle



In [ ]:
# PARAMETERS

DEVICE = "cuda"
SEED = 42

rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True



In [ ]:
# UTILS FUNCTIONS

def load_dataframe( dataset_file : str):
    return pd.read_csv(dataset_file)


def find_null_or_empty_records( dataframe: pd.DataFrame):
    n = len(dataframe)
    for index, row in dataframe.iterrows():
        print_progress_bar(index/n)
        # Controlla se ci sono valori nulli o vuoti nel record
        if row.isnull().any() or any(map(lambda x: x == '', row)):
            # Stampa il record
            print(f"Record con valori nulli o vuoti:\n{row}\n")

def print_progress_bar(percentuale, lunghezza_barra=20):
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% completo")
    sys.stdout.flush()


def compute_kind_inconsistence(dataframe):
    return {"inconsistent orig balance": len(dataframe.query('abs(oldbalanceOrg - newbalanceOrig) != amount'))/len(dataframe),
            "inconsistent dest balance": len(dataframe.query('abs(oldbalanceDest - newbalanceDest) != amount'))/len(dataframe),
            "zero cash transaction": len(dataframe.query('amount == 0 '))/len(dataframe),
            "self-transaction": len(dataframe.query('nameOrig == nameDest'))/len(dataframe)
            }

def plot_histogram(to_plot):


    # Converti il dizionario in un array di valori
    values = list(to_plot.values())

    # Crea un istogramma
    fig = go.Figure(data=[go.Bar(x=list(to_plot.keys()), y=values)])

    # Mostra l'istogramma
    fig.show()



def plot_categories(dataframe):
    # Calcola la frequenza di ogni categoria nella colonna 'type'
    counts = dataframe['type'].value_counts().reset_index()

    # Rinomina le colonne
    counts.columns = ['type', 'count']

    counts['count'] = counts['count'] / counts['count'].sum()

    # Crea l'istogramma con Plotly Express
    fig = px.bar(counts, x='type', y='count', title='Istogramma delle categorie nella colonna "type"')

    # Mostra il plot
    fig.show()




In [ ]:
class FraudDetectionDataset(Dataset):

    def __init__(self,dataset_file : str):
        dataframe = load_dataframe(dataset_file)
        dataframe["isFraud"] = dataframe["isFraud"].astype(bool)
        dataframe["isFlaggedFraud"] = dataframe["isFlaggedFraud"].astype(bool)
        self.raw_data = dataframe



    def analize_data(self,find_empty_records = True):
        print("----HEAD----")
        print(self.raw_data.head())
        print("----INFO----")
        print(self.raw_data.info())
        print("----DESCRIBE----")
        print(self.raw_data.describe())
        if find_empty_records:
          find_null_or_empty_records(self.raw_data)

    def extract_inconsistent_transactions(self):
        condiction = "abs(oldbalanceOrg - newbalanceOrig) != amount | abs(oldbalanceDest - newbalanceDest) != amount | amount == 0 | nameOrig == nameDest"

        return self.raw_data.query(condiction)


    def remove_inconsistent(self):
        self.raw_data = self.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) == abs(oldbalanceDest - newbalanceDest) & amount != 0")




    def balanced_batch_generator(X, y, batch_size, minority_class_percent=0.5):

        majority_class_indices = np.where(y == 0)[0]
        minority_class_indices = np.where(y == 1)[0]

        minority_class_batch_size = int(batch_size * minority_class_percent)

        while True:

            sampled_majority_indices = np.random.choice(majority_class_indices, size=batch_size - minority_class_batch_size, replace=False)

            sampled_minority_indices = np.random.choice(minority_class_indices, size=minority_class_batch_size, replace=False)

            batch_indices = np.concatenate([sampled_majority_indices, sampled_minority_indices])

            batch_indices = shuffle(batch_indices)
            X_batch, y_batch = X[batch_indices], y[batch_indices]

            yield X_batch, y_batch

    def collate(self):
      # trasformare batch in grafo





In [ ]:
dataset = FraudDetectionDataset("PS_20174392719_1491204439457_log.csv")
inconsistent_data = dataset.extract_inconsistent_transactions()

In [ ]:
d = compute_kind_inconsistence(inconsistent_data)
plot_histogram(d)
names_distinct = set(dataset.raw_data["nameOrig"]).union(set(dataset.raw_data["nameDest"]))

In [ ]:
dataset.analize_data(find_empty_records=False)
print("inconsistent fraud")
print(len(inconsistent_data.query('isFraud == 1'))/len(inconsistent_data))
print("inconsistent not fraud")
print(len(inconsistent_data.query('isFraud == 0'))/len(inconsistent_data))
print("given > gotten")
print(len(dataset.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) > abs(oldbalanceDest - newbalanceDest)"))/len(inconsistent_data))
print("given < gotten")
print(len(dataset.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) < abs(oldbalanceDest - newbalanceDest)"))/len(inconsistent_data))
print("distinct origin name")
print(dataset.raw_data["nameOrig"].nunique())
print("distinct names")
print(len(names_distinct))
print("isFlaggedFraud and isFraud")
print(len(dataset.raw_data.query("isFlaggedFraud & isFraud"))/len(dataset.raw_data.query("isFraud")))
plot_categories(dataset.raw_data)
